In [1]:
import urllib.request
import json
from tqdm import tnrange, tqdm_notebook
import time

In [10]:
def youtubeLinkFromPvList(list):
  for i in list:
    if 'youtu.be' in i['url']:
      return i['url']
  return ''

def genLinkForGet(ind):
  return 'https://vocadb.net/api/songs?start=' \
    + str(ind) \
    + '&getTotalCount=true&maxResults=50&fields=pvs&lang=Default&nameMatchMode=Auto&sort=RatingScore&childTags=false&artistParticipationStatus=Everything&onlyWithPvs=false'

In [11]:
finalPvList = []

totalSongs = 1000
maxPerReq = 50


In [12]:
for i in tqdm_notebook(range(int(totalSongs / maxPerReq)), desc='outer loop'):
  link = genLinkForGet(i * maxPerReq)
  contents = json.loads(urllib.request.urlopen(link).read())

  for item in tqdm_notebook(contents['items'], desc='inner loop', leave=False):
    ytLink = youtubeLinkFromPvList(item['pvs'])
    if ytLink:
      finalPvList.append(ytLink)
outfile = open('pvlist.txt', 'w')
for item in finalPvList:
  outfile.write("%s\n" % item)

In [2]:
infile = open('pvlist.txt', 'r')

In [3]:
finalPvList = infile.readlines()

In [6]:
import subprocess
from threading import Thread

from queue import Queue
limit = 3

def worker(queue):
    for link in iter(queue.get, None):
        try:
            downloadCmd = 'youtube-dl -o "vocaloid/%(title)s.%(ext)s" --download-archive downloaded.txt --no-post-overwrites -ciwx --audio-format mp3 ' + link
            subprocess.check_call(downloadCmd, stderr=subprocess.STDOUT, shell=True)
        except:
            continue


In [7]:
downloadQueue = Queue()
threads = [Thread(target=worker, args=(downloadQueue,)) for _ in range(limit)]
for link in finalPvList:  # feed commands to threads
    downloadQueue.put_nowait(link)

for t in threads: # start workers
    t.daemon = True
    t.start()

for _ in threads: downloadQueue.put(None) # signal no more commands
for t in threads: t.join()    # wait for completion